In [5]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import Parallel, delayed
from tqdm import tqdm
import os

def evaluate_combo(X, y, combo, cv_splits=5):
    """
    Evaluate a single variable combination with k-fold CV.
    Fits OLS on training folds, checks p-values, computes RMSE and adjusted R² on each fold.
    Returns metrics if all predictor p-values < 0.05 in each fold; else None.
    """
    X_sub = X[list(combo)]
    X_sub = sm.add_constant(X_sub)
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

    rmses = []
    adj_r2s = []
    p_values_valid = True
    coefs = None
    pvals = None

    for train_idx, test_idx in kf.split(X_sub):
        X_train, X_test = X_sub.iloc[train_idx], X_sub.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        try:
            model = sm.OLS(y_train, X_train).fit()
        except Exception as e:
            # Model failed to fit, discard this combo
            p_values_valid = False
            break

        # Check all predictors (except const) p-values < 0.05
        if (model.pvalues.drop('const') >= 0.05).any():
            p_values_valid = False
            break

        preds = model.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, preds))
        rmses.append(rmse)
        adj_r2s.append(model.rsquared_adj)

        # Keep last fold’s coefficients and p-values for reporting
        coefs = model.params.to_dict()
        pvals = model.pvalues.to_dict()

    if not p_values_valid:
        return None

    return {
        'variables': combo,
        'mean_cv_rmse': sum(rmses) / len(rmses),
        'mean_cv_adj_r2': sum(adj_r2s) / len(adj_r2s),
        'coefficients': coefs,
        'pvalues': pvals
    }

def process_target(X, y, target_name, n_jobs=4, max_combination_size=None):
    """
    Process all variable combinations for a single target variable.
    Saves intermediate results after each combination size.
    Returns a DataFrame of results.
    """
    results = []
    max_size = max_combination_size or len(X.columns)

    for size in range(1, max_size + 1):
        combos = list(combinations(X.columns, size))
        print(f"Processing target '{target_name}' - combinations of size {size} ({len(combos)} combos)")
        
        batch_results = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_combo)(X, y, combo) for combo in tqdm(combos, desc=f"Size {size} combos")
        )

        # Filter out None results (failed p-value or fit)
        batch_results = [res for res in batch_results if res is not None]

        # Add target name for each result
        for res in batch_results:
            res['target'] = target_name

        results.extend(batch_results)

        # Save checkpoint after each size
        if results:
            df = pd.DataFrame(results)
            checkpoint_path = f'results_{target_name}_up_to_size_{size}.parquet'
            df.to_parquet(checkpoint_path, index=False)
            print(f"Checkpoint saved: {checkpoint_path}")

    if results:
        return pd.DataFrame(results)
    else:
        return pd.DataFrame()  # empty if no valid models

def save_results(df, filename_csv='ols_model_results.csv', filename_parquet='ols_model_results.parquet'):
    """
    Save the results DataFrame to CSV or Parquet depending on size.
    Prints status messages.
    """
    if df.empty:
        print("No valid models found.")
        return

    df.to_csv(filename_csv, index=False)
    file_size = os.path.getsize(filename_csv)
    size_mb = file_size / (1024*1024)

    if size_mb > 5:  # 5 MB threshold for switching to Parquet
        df.to_parquet(filename_parquet, index=False)
        os.remove(filename_csv)
        print(f"Results saved to {filename_parquet} ({size_mb:.2f} MB)")
    else:
        print(f"Results saved to {filename_csv} ({size_mb:.2f} MB)")

def main():
    # Load data
    X = pd.read_excel(r"C:\Users\Administrator\Desktop\dataforcode\sent2(GYr).xlsx")
    Y = pd.read_csv(r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sparameters.csv.")

    all_results = []

    # You can adjust n_jobs based on your CPU cores
    n_jobs = 6
    max_comb_size = None  # or set to e.g. 5 to limit computation

    for target in Y.columns:
        print(f"\n=== Starting processing for target: {target} ===")
        y = Y[target]
        df_target_results = process_target(X, y, target, n_jobs=n_jobs, max_combination_size=max_comb_size)
        if not df_target_results.empty:
            all_results.append(df_target_results)
        else:
            print(f"No valid models found for target '{target}'.")

    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        save_results(final_df)
    else:
        print("No valid models found for any target.")

    print("All computations completed.")

if __name__ == "__main__":
    main()



=== Starting processing for target: pH ===
Processing target 'pH' - combinations of size 1 (10 combos)


Size 1 combos: 100%|██████████| 10/10 [00:00<00:00, 358.10it/s]


Processing target 'pH' - combinations of size 2 (45 combos)


Size 2 combos: 100%|██████████| 45/45 [00:00<00:00, 2148.38it/s]


Checkpoint saved: results_pH_up_to_size_2.parquet
Processing target 'pH' - combinations of size 3 (120 combos)


...

Checkpoint saved: results_TDS_up_to_size_8.parquet
Processing target 'TDS' - combinations of size 9 (10 combos)


Size 9 combos: 100%|██████████| 10/10 [00:00<?, ?it/s]


Checkpoint saved: results_TDS_up_to_size_9.parquet
Processing target 'TDS' - combinations of size 10 (1 combos)


Size 10 combos: 100%|██████████| 1/1 [00:00<00:00, 1041.29it/s]


Checkpoint saved: results_TDS_up_to_size_10.parquet
Results saved to ols_model_results.csv (0.01 MB)
All computations completed.


In [1]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import Parallel, delayed
from tqdm import tqdm
import os

def evaluate_combo(X, y, combo, cv_splits=5):
    """
    Evaluate a single variable combination with k-fold CV.
    Fits OLS on training folds, checks p-values, computes RMSE and adjusted R² on each fold.
    Returns metrics if all predictor p-values < 0.05 in each fold; else None.
    """
    X_sub = X[list(combo)]
    X_sub = sm.add_constant(X_sub)
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

    rmses = []
    adj_r2s = []
    p_values_valid = True
    coefs = None
    pvals = None

    for train_idx, test_idx in kf.split(X_sub):
        X_train, X_test = X_sub.iloc[train_idx], X_sub.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        try:
            model = sm.OLS(y_train, X_train).fit()
        except Exception as e:
            # Model failed to fit, discard this combo
            p_values_valid = False
            break

        # Check all predictors (except const) p-values < 0.05
        if (model.pvalues.drop('const') >= 0.05).any():
            p_values_valid = False
            break

        preds = model.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, preds))
        rmses.append(rmse)
        adj_r2s.append(model.rsquared_adj)

        # Keep last fold’s coefficients and p-values for reporting
        coefs = model.params.to_dict()
        pvals = model.pvalues.to_dict()

    if not p_values_valid:
        return None

    return {
        'variables': combo,
        'mean_cv_rmse': sum(rmses) / len(rmses),
        'mean_cv_adj_r2': sum(adj_r2s) / len(adj_r2s),
        'coefficients': coefs,
        'pvalues': pvals
    }

def process_target(X, y, target_name, n_jobs=4, max_combination_size=None):
    """
    Process all variable combinations for a single target variable.
    Saves intermediate results after each combination size.
    Returns a DataFrame of results.
    """
    results = []
    max_size = max_combination_size or len(X.columns)

    for size in range(1, max_size + 1):
        combos = list(combinations(X.columns, size))
        print(f"Processing target '{target_name}' - combinations of size {size} ({len(combos)} combos)")
        
        batch_results = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_combo)(X, y, combo) for combo in tqdm(combos, desc=f"Size {size} combos")
        )

        # Filter out None results (failed p-value or fit)
        batch_results = [res for res in batch_results if res is not None]

        # Add target name for each result
        for res in batch_results:
            res['target'] = target_name

        results.extend(batch_results)

        # Save checkpoint after each size
        if results:
            df = pd.DataFrame(results)
            checkpoint_path = f'results_{target_name}_up_to_size_{size}.parquet'
            df.to_parquet(checkpoint_path, index=False)
            print(f"Checkpoint saved: {checkpoint_path}")

    if results:
        return pd.DataFrame(results)
    else:
        return pd.DataFrame()  # empty if no valid models

def save_results(df, filename_csv='ols_model_results.csv', filename_parquet='ols_model_results.parquet'):
    """
    Save the results DataFrame to CSV or Parquet depending on size.
    Prints status messages.
    """
    if df.empty:
        print("No valid models found.")
        return

    df.to_csv(filename_csv, index=False)
    file_size = os.path.getsize(filename_csv)
    size_mb = file_size / (1024*1024)

    if size_mb > 5:  # 5 MB threshold for switching to Parquet
        df.to_parquet(filename_parquet, index=False)
        os.remove(filename_csv)
        print(f"Results saved to {filename_parquet} ({size_mb:.2f} MB)")
    else:
        print(f"Results saved to {filename_csv} ({size_mb:.2f} MB)")

def main():
    # Load data
    X = pd.read_excel(r"C:\Users\Administrator\Desktop\dataforcode\LS8,9(GYr).xlsx")
    Y = pd.read_csv(r"C:\Users\Administrator\Desktop\code\band to band\landsat8,9\lparameters.csv.")

    all_results = []

    # You can adjust n_jobs based on your CPU cores
    n_jobs = 6
    max_comb_size = None  # or set to e.g. 5 to limit computation

    for target in Y.columns:
        print(f"\n=== Starting processing for target: {target} ===")
        y = Y[target]
        df_target_results = process_target(X, y, target, n_jobs=n_jobs, max_combination_size=max_comb_size)
        if not df_target_results.empty:
            all_results.append(df_target_results)
        else:
            print(f"No valid models found for target '{target}'.")

    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        save_results(final_df)
    else:
        print("No valid models found for any target.")

    print("All computations completed.")

if __name__ == "__main__":
    main()



=== Starting processing for target: pH ===
Processing target 'pH' - combinations of size 1 (8 combos)


Size 1 combos: 100%|██████████| 8/8 [00:00<00:00, 235.92it/s]


Processing target 'pH' - combinations of size 2 (28 combos)


Size 2 combos: 100%|██████████| 28/28 [00:00<00:00, 2004.62it/s]


Processing target 'pH' - combinations of size 3 (56 combos)


...

Processing target 'TDS' - combinations of size 7 (8 combos)


Size 7 combos: 100%|██████████| 8/8 [00:00<00:00, 7994.86it/s]


Processing target 'TDS' - combinations of size 8 (1 combos)


Size 8 combos: 100%|██████████| 1/1 [00:00<?, ?it/s]

No valid models found for target 'TDS'.
Results saved to ols_model_results.csv (0.00 MB)
All computations completed.
